# Diplomatura en Ciencias de Datos, Aprendizaje Automático y sus Aplicaciones

Autores: Matías Oria, Antonela Sambuceti, Pamela Pairo, Benjamín Ocampo

## Mejores 3 Modelos de la Competencia
Se trabajó sobre los siguientes modelos:

- `xgb`: XGBoost.
- `lgr`: Regresión logistica.
- `svm`: Support Vector Machines.
- `random_forest`: Random Forest.
- `dtree`: Decision Tree.
- `sgd`: SGD Classifier para regresión logistica.
- `nnet`: Neural Networks (Sklearn).
- `nnet_keras`: Neural Networks (Keras).
- `naive_bayes`: Naive Bayes.

De los cuales los `xgb`, `svm`, y `rndforest` lograron los mejores resultados
para el conjunto de *test* público en la competencia de Kaggle.

La implementación y busqueda de hiperparametros del resto de modelos puede
encontrarse en una notebook dedicada para cada uno.

### Pipeline
El pipeline planteado durante el proceso de aprendizaje consiste de 3 capas.

- Discretización para las variables continuas `Age`, y `AnnualIncome`.
- Códificación one-hot para el resto de variables categoricas.
- Modelo.

Donde las primeras dos son de preprocesamiento, mientras que la tercera es el
predictor utilizado.

In [39]:
from utils import (make_pipeline, save_predictions, X_train_total,
                   y_train_total, X_train, X_val, y_train, y_val, X_test,
                   df_test, seed, DIR_SUBMISSIONS)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from os import makedirs
import pandas as pd

### 1º XGBoost
Para este modelo se optó por realizar una busqueda de hiperparámetros
utilizando todas las filas y columnas del conjunto de entrenamiento por medio
de una Busqueda Bayesiana (`Bayesian Search`) donde se especifica un rango o
espacio sobre los parametros que se desean encontrar. Este espacio se recorre
de manera apropiada recordando iteraciones pasadas de busqueda hasta encontrar
el modelo óptimo. Algo a recalcar es que no solamente se ajustó la signature
del clasificador, si no que también se intentó encontrar distintas
discretizaciones posibles durante el preprocesamiento. 

Resultado en Kaggle f1-score: 0.80898

In [2]:
from xgboost import XGBClassifier

xgboost = make_pipeline(XGBClassifier())
params_bayes = {
    "preprocessor__discretizer__n_bins": Integer(2, 25),
    "model__objective": Categorical(["binary:logistic"]),
    "model__n_estimators": Integer(10, 500),
    "model__gamma": Real(1e-6, 1),
    "model__max_depth": Integer(4, 20),
    "model__learning_rate": Real(1e-4, 1),
    "model__alpha": Real(1e-4, 1),
    "model__booster": Categorical(["gbtree", "gblinear", "dart"]),
    "model__colsample_bytree": Real(.5, 1),
    "model__subsample": Real(.6, 1),
    "model__eval_metric": Categorical(["logloss"]),
    "model__use_label_encoder": Categorical([False]),
}
opt = BayesSearchCV(xgboost, params_bayes, scoring="f1")
opt.fit(X_train_total, y_train_total)
save_predictions(opt.best_estimator_, "xgb.csv")
opt.best_params_

/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/pyth

[10:48:42] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:48:42] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:48:42] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.

/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/pyth

[10:48:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:48:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:48:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.

/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/pyth

...

[10:52:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.

/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/pyth

[10:53:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:53:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.cc:573: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:53:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1631904754241/work/src/learner.

/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/benjamin/miniconda3/envs/diplodatos-supervised/lib/pyth

OrderedDict([('model__alpha', 0.6066724149261147),
             ('model__booster', 'gbtree'),
             ('model__colsample_bytree', 1.0),
             ('model__eval_metric', 'logloss'),
             ('model__gamma', 1.0),
             ('model__learning_rate', 1.0),
             ('model__max_depth', 4),
             ('model__n_estimators', 500),
             ('model__objective', 'binary:logistic'),
             ('model__subsample', 1.0),
             ('model__use_label_encoder', False),
             ('preprocessor__discretizer__n_bins', 16)])

### 2º SVM
El segundo modelo consta de una busqueda de hiperparametros por medio de una
Grid Search recorriendo sobre 4 kernels posibles, `linear`, `sigmoid`, `poly`,
`rbf`. A diferencia de `xgb`, en este caso se optó por dividir los datos en
train y validación para obtener métricas del modelo sobre este último conjunto.
Una vez realizada la busqueda de hiperparametros sobre el conjunto de datos de
entrenamiento, se utilizan los mejores hiperparametros para realizar un último
ajuste con `train` + `validation`.

Resultado en Kaggle f1-score: 0.80459

In [3]:
from sklearn.svm import SVC
svm = make_pipeline(SVC())

params = {
    "model__kernel": ["linear", "sigmoid", "poly", "rbf"],
    "model__gamma": ["scale", "auto"],
    "model__degree": [2, 3, 4],
    "model__coef0": [.001, .01, 0, 1],
    "model__tol": [1e-2, 1e-3, 1e-4],
    "model__C": [1, 0.1, 0.01, 0.001, 0.0001, 10],
    "model__class_weight": ["balanced"]
}
clf = GridSearchCV(svm, param_grid=params, scoring="f1")
clf.fit(X_train, y_train)
y_val_pred = clf.best_estimator_.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       192
           1       0.90      0.59      0.72       106

    accuracy                           0.83       298
   macro avg       0.86      0.78      0.80       298
weighted avg       0.84      0.83      0.82       298



In [4]:
# Refit with validation data
best_model = make_pipeline(
    SVC(
        **{
            key.removeprefix("model__"): value
            for key, value in clf.best_params_.items()
        }))
best_model.fit(X_train_total, y_train_total)
save_predictions(best_model, "svm.csv")

### 3º Random Forest
En este último caso, se optó por utilizar Busqueda Bayesiana dividiendo los datos de entrenamiento y se obtuvo el score F1 mas alto eliminando la variable `GraduateOrNot`-

Resultado en Kaggle f1-score: 0.80459

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train_total = X_train_total.drop(["GraduateOrNot"], axis=1)
X_train = X_train.drop(["GraduateOrNot"], axis=1)
X_test = X_test.drop(["GraduateOrNot"], axis=1)
X_val = X_val.drop(["GraduateOrNot"], axis=1)

In [15]:
vars_to_ohe = X_train_total.select_dtypes(include=['object']).columns

vars_to_scale = ["Age", "AnnualIncome", "FamilyMembers"]

preprocessor = ColumnTransformer(
    [("encoder", OneHotEncoder(), vars_to_ohe),
     ("scaler", StandardScaler(), vars_to_scale)],
    remainder="passthrough")

In [18]:
pipe_rfc = Pipeline([
    ("preprocessor", preprocessor),
    ("rfc", RandomForestClassifier(random_state=seed))
])

In [22]:
params_rfc = {
    "rfc__max_depth": Integer(5, 40),
    "rfc__criterion":Categorical(['gini', 'entropy']),
    "rfc__min_samples_split": Integer(3,20),
    "rfc__min_samples_leaf":Integer(1,25),
    "rfc__bootstrap": Categorical([False]),
    "rfc__n_estimators": Integer(20, 150)
}

opt = BayesSearchCV(pipe_rfc, params_rfc, scoring="f1", cv=3)
opt.fit(X_train, y_train)

BayesSearchCV(cv=3,
              estimator=Pipeline(steps=[('preprocessor',
                                         ColumnTransformer(remainder='passthrough',
                                                           transformers=[('encoder',
                                                                          OneHotEncoder(),
                                                                          Index(['Employment Type', 'FrequentFlyer', 'EverTravelledAbroad'], dtype='object')),
                                                                         ('scaler',
                                                                          StandardScaler(),
                                                                          ['Age',
                                                                           'AnnualIncome',
                                                                           'FamilyMembers'])])),
                                        ('rfc',
          

In [23]:
opt.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder', OneHotEncoder(),
                                                  Index(['Employment Type', 'FrequentFlyer', 'EverTravelledAbroad'], dtype='object')),
                                                 ('scaler', StandardScaler(),
                                                  ['Age', 'AnnualIncome',
                                                   'FamilyMembers'])])),
                ('rfc',
                 RandomForestClassifier(bootstrap=False, max_depth=40,
                                        min_samples_leaf=9,
                                        min_samples_split=20, n_estimators=134,
                                        random_state=0))])

In [25]:
y_val_pred = opt.best_estimator_.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88       192
           1       0.91      0.58      0.71       106

    accuracy                           0.83       298
   macro avg       0.86      0.78      0.80       298
weighted avg       0.85      0.83      0.82       298



In [40]:
def save_predictions(model, filename):
    test_id = df_test["Customer"]
    test_pred = model.predict(df_test.drop(columns=["Customer", "GraduateOrNot"]))

    submission = pd.DataFrame(list(zip(test_id, test_pred)),
                              columns=["Customer", "TravelInsurance"])
    
    makedirs(DIR_SUBMISSIONS, exist_ok=True)
    submission.to_csv(f"{DIR_SUBMISSIONS}/{filename}",
                      header=True,
                      index=False)

In [41]:
save_predictions(opt.best_estimator_, "random_forest.csv")